# Reverse Engineering the Game

Game client is developed with Unity so the code can be decompiled with ILSpy. We are interested in digging **Assembly-CSharp.dll**, which contain game code including server-client communication. Server IP can be set in the **ServerConfig.xml** file located in the fame folder. We can use 127.0.0.1 for development purposes.

## Starting the Game

After the splash screen we see the following statuses:

1. Validating Connectivity...
2. Waiting for Connection...
3. Server Connection Closed! - Reconnecting
4. Waiting for Connection... (infinite)

Let's create a basic tcp (not http) server and see what we can get.

```js
const net = require('net');
const server = net.createServer();
server.listen(8182, '127.0.0.1');
server.on('connection', socket => {
  socket.on('data', data => {
    console.log(`str: ${data}`);
    console.log('hex:', data, data.byteLength, 'bytes', '\n');
  });
});
```

- Spoilers: the first message from the client is 22 bytes: ```§ �\♦m♀♀↑AmazingWorld```
- hex: <15 20 c2 5c 04 6d 0c 0c 18 41 6d 61 7a 69 6e 67 57 6f 72 6c 64 00>

## Binary World

In [1]:
def print_i(i_arr):
    print('bit   int    hex           bin       char')
    for bit, i in enumerate(i_arr):
        int_str = str(i).ljust(3)
        hex_str = hex(i)[2:].zfill(2)
        bin_str = bin(i)[2:].zfill(8)
        bin_str = str(bit * 8).rjust(3) + ': ' + bin_str[:4] + ' ' + bin_str[4:]
        chr_str = chr(i) if i <= 127 else ''
        bit = str(bit + 1).ljust(2)
        print(bit, int_str, hex_str, bin_str, chr_str, sep='    ')

src_hex = "15 20 c2 5c 04 6d 0c 0c 18 41 6d 61 7a 69 6e 67 57 6f 72 6c 64 00".split()
src_int = list(map(lambda i: int(i, 16), src_hex))
print_i(src_int)

bit   int    hex           bin       char
1     21     15      0: 0001 0101    
2     32     20      8: 0010 0000     
3     194    c2     16: 1100 0010    
4     92     5c     24: 0101 1100    \
5     4      04     32: 0000 0100    
6     109    6d     40: 0110 1101    m
7     12     0c     48: 0000 1100    
8     12     0c     56: 0000 1100    
9     24     18     64: 0001 1000    
10    65     41     72: 0100 0001    A
11    109    6d     80: 0110 1101    m
12    97     61     88: 0110 0001    a
13    122    7a     96: 0111 1010    z
14    105    69    104: 0110 1001    i
15    110    6e    112: 0110 1110    n
16    103    67    120: 0110 0111    g
17    87     57    128: 0101 0111    W
18    111    6f    136: 0110 1111    o
19    114    72    144: 0111 0010    r
20    108    6c    152: 0110 1100    l
21    100    64    160: 0110 0100    d
22    0      00    168: 0000 0000     


In [2]:
print('It looks like the first byte contains message size:', src_int[0], 'bytes')
print('Last byte contain nothing:', src_int[-1])

It looks like the first byte contains message size: 21 bytes
Last byte contain nothing: 0


## The fuck is this?

- idk, lol. looks like serialized data structure with 'AmazingWorld' string
- let's dig into the code by searching "validating connectivity" or "AmazingWorld"

| Action                        | Where                                  | What happens                                                                                 |
| ----------------------------- | -------------------------------------- | -------------------------------------------------------------------------------------------- |
| 1. Server check start         | AkamaiServerCheck.Start()              | State = WaitingForServerCheck                                                                |
| 2. Server check cycle         | AkamaiServerCheck.Update()             | "Validating Connectivity..."                                                                 |
|                               |                                        | To pass server check, ServerCheckSuccess() should be called                                  |
| 3. Create Session             | AkamaiServerCheck.CheckClientVersion() | State = WaitingForConnect                                                                    |
|                               |                                        | **Call**: ClientManager.Instance.CreateSession()                                             |
|                               |                                        | **Callback**: AkamaiServerCheck.OnConnect()                                                  |
| 4. Send 'AmazingWorld'        | AkamaiServerCheck.OnConnect()          | **Call**: ClientManager.Instance.session.SendMessage()                                       |
|                               |                                        | **Args**: serviceClass = 18, messageType = 566                                               |
|                               |                                        | **Args**: GSFGetClientVersionInfoSvc.GSFRequest with 'AmazingWorld' as clientName            |
|                               |                                        | **Callbacks**: GetClientVersionInfoResponseHandler() and GetClientVersionInfoErrorHandler()  |
| 5. Game Spark Framework?      | GSFSession.SendMessage()               | Wraps request into the **new GSFRequestMessage(serviceClass, messageType, request)**         |
| 6. Wrapping GSFRequestMessage | GSFRequestMessage.GSFRequestMessage()  | Timeout, discardable and type attributes added                                               |
| 7. Continue to send message   | GSFSession._SendMessage()              | Add message to the **responseHandlers** and **outRequestQueue**                              |
| 8. Write loop ?               | GSFSession.DoWriteLoop()               | I do not know the source event, but this is the only place where **outRequestQueue** is used |
| 9. Write message from queue   | GSFSession.WriteMessage()              | Let's slow down a little bit, because binary stuff incoming!                                 |

1. It was at this moment that he knew he f*cked up
2. I created a debug project to trace message encoding process, see ./Amazing-Tester/Amazing-Tester.sln
3. I wrote out, cleaned and commented all the necessary code parts for message serialization and bit encoding

```cs
class GSFSession {
    void WriteMessage(GSFMessage call) {
        MemoryStream memoryStream = new MemoryStream();
        BinaryWriter writer = new BinaryWriter(memoryStream);
        gSFBitProtocolCodec.WriteMessage(call, writer); // Converting GSFMessage into bytes
        // ...
        mProtocol.WriteLength((uint)num, writer); // Write message size in bytes
        writer.Write(array); // Write message
        writer.Write((byte)0); // Write 0 byte
        // ...
    }
}
```

The message indeed consists of message size, message content and a zero byte. We are interested in encoding process, so let's dig into the gSFBitProtocolCodec.WriteMessage method.

```cs
class GSFBitProtocolCodec {
    void WriteMessage(GSFMessage msg, BinaryWriter writer) {
        GSFIProtocolOutput gSFIProtocolOutput = new BitOutput();
        gSFIProtocolOutput.OutputWriter = writer;
        gSFIProtocolOutput.Write(msg); // Converting GSFMessage into bytes array
        gSFIProtocolOutput.CommitToWriter();
    }
}
```

```cs
class BitOutput {
    GSFBitStream bitStream = new GSFBitStream();

    Write(GSFIExternalizable o) {
        if (!bitStream.Put(o == null)) { // if o != null
            o.SerializeMembers(ProtocolType.Bit, this);
        }
    }

    Write(int i) {
        bitStream.PutIntCompressed(i);
    }

    void Write(string s) {
        if (s != null)
        {
            byte[] array = GSFTextUtils.ToUtf8Bytes(s);
            int num = array.Length;
            bitStream.PutIntCompressed(num);
            if (0 < num) {
                bitStream.PutBytesAligned(array);
            }
        }
        else {
            bitStream.PutIntCompressed(-1);
        }
    }
    
}
```

```cs
class MessageHeader : GSFIExternalizable {
    void SerializeMembers(ProtocolType protocol, GSFIProtocolOutput output) {
        output.Write(flags);
        output.Write(svcClass);
        output.Write(msgType);
        if (IsService) {
            output.Write(requestId);
        }
        if (IsRequest) {
            output.Write(logCorrelator);
        }
    }
}

class GSFGetClientVersionInfoSvc : GSFClientService {
    class GSFRequest : GSFService.GSFRequest {
        public string clientName; // AmazingWorld
        
        void SerializeMembers(ProtocolType protocol, GSFIProtocolOutput output) {
            base.SerializeMembers(protocol, output);
            output.Write(clientName);
        }
    }
}

class GSFRequestMessage : GSFMessage {
    void SerializeMembers(ProtocolType protocol, GSFIProtocolOutput output) {
        base.SerializeMembers(protocol, output); // GSFMessage
        output.Write(body); // GSFGetClientVersionInfoSvc.GSFRequest
    }
}

class GSFMessage : GSFTransportObject {
    void SerializeMembers(ProtocolType protocol, GSFIProtocolOutput output) {
        base.SerializeMembers(protocol, output); // GSFTransportObject (blank)
        output.Write(header); // MessageHeader -> Write(GSFIExternalizable o)
    }
}
```

1. When object is not a primitive type, write 0 and serialize it
2. SerializeMembers methods send attributes back to the BitOutput encoder
3. Going to the lowest Message level with base.SerializeMembers (MessageHeader)
4. Write(flags) using bitStream.PutIntCompressed(i);

```cs
class GSFBitStream {
    IntCompressor noCompress = new IntBytes();
    IntCompressor compressor = new IntModeX(); // GetCompressor(90);

    byte[] BM = new byte[8] { 128, 64, 32, 16, 8, 4, 2, 1 };
    byte[] BML = new byte[8] { 0, 128, 192, 224, 240, 248, 252, 254 };
    
    int IndexByte(int index) => off + (index >> 3); // Buffer index according to the position (1 for 8 bits)
    int IndexBit(int index) => index & 7; // Bit index accordint to the position (from 0 to 7)

    GSFBitStream() {
        off = 0; pos = 0; mark = -1;
        buf = new byte[1024]; // new byte[IndexByte(8199)];
        lim = 8192; cap = 8192; // buf.Length << 3;
    }

    bool Put(bool bit) {
        // bit = GSFMessage == null
        if (bit) { buf[IndexByte(pos)] |= BM[IndexBit(pos)]; } // Set current bit (1)
        else { buf[IndexByte(pos)] &= (byte)(~BM[IndexBit(pos)]); }
        pos++; return bit;
    }

    void PutByte(byte b) {
        int num = IndexByte(pos);
        int num2 = IndexBit(pos);
        
        if (num2 == 0) {
            buf[num] = b; // Write the whole byte if position is in the beginning
        }
        else {
            buf[num] &= BML[num2];
            // Fill current bit with shifting to take only first part
            buf[num] |= b >> IndexBit(pos);
            // 0xFF >> 8 - IndexBit(pos) mask to get rest bits that will go to the next byte start
            buf[num + 1] = (b & (0xFF >> 8 - IndexBit(pos))) << (8 - IndexBit(pos)); 
            // int num3 = (0xFF & b) << 8 - num2;
            // buf[num] |= (byte)(num3 >> 8);
            // buf[num + 1] = (byte)(BML[num2] & num3);
        }
        
        pos += 8;
    }

    void PutIntCompressed(int val) {
        compressor.Put(this, val, 4);
    }    
}
```

1. IndexByte returns current byte index inside the buffer
2. IndexBit return current bit index inside the current byte
3. Put enables or Disables current bit and increments current posision
4. PutByte writes whole byte or divides value between current and next bytes

In [3]:
print_i([42]) # For example we want to write value 42
position = 21 # Our current position is 21

bit   int    hex           bin       char
1     42     2a      0: 0010 1010    *


In [4]:
index_byte = position >> 3
index_bit = position & 7
print('index_byte:', index_byte)
print('index_bit:', index_bit)

index_byte: 2
index_bit: 5


In [5]:
# Current index bit is 5, meaning we have 2 bits available in the current byte
# Megning we should move value >> 5 to get 0000 0001 in the current and 0101 0000 in the next one
current_bit = 42 >> index_bit
next_bit = (42 & (0xFF >> 8 - index_bit)) << (8 - index_bit)
print_i([42, current_bit, next_bit])

bit   int    hex           bin       char
1     42     2a      0: 0010 1010    *
2     1      01      8: 0000 0001    
3     80     50     16: 0101 0000    P


```cs
class IntModeX {
    long[] _min = new long[9] { -8L, -128L, -32768L, -8388608L, -2147483648L, -549755813888L, -140737488355328L, -140737488355328L, -9223372036854775808L };
    long[] _max = new long[9] { 7L, 127L, 32767L, 8388607L, 2147483647L, 549755813887L, 140737488355327L, 140737488355327L, 9223372036854775807L };

    void Put(GSFBitStream bs, long val, int w) {
        int i = 0;

        // Increment i for every 4, 8 and 16 bits if value requires
        for (; i < w && (val < _min[i] || val > _max[i]); i++) { }
        // 0: (-8, 7), 1: (-128, 127), 2: (-32768, 32767), 3: (-8388608, 8388607)

        if (bs.Put(i < w)) { // (1)
            // (1) for every extra 4 bits + (0)
            for (int j = 0; bs.Put(j < i) && j < w; j++) { }
        }

        if (i > 0) {
            int num = (i - 1) * 8; // 1: 0, 2: 8, 3: 16
            while (num >= 0) {
                bs.PutByte((byte)(0xFFu & ((uint)val >> num)));
                num -= 8;
            }
        }
        else {
            int num2 = 8; // Fill next 4 bytes with integer value
            while (num2 > 0) {
                bs.Put((num2 & val) != 0);
                num2 >>= 1;
            }
        }
    }
}
```

## "AmazingWorld" Encoding Process

In [6]:
print_i(src_int[1:])

bit   int    hex           bin       char
1     32     20      0: 0010 0000     
2     194    c2      8: 1100 0010    
3     92     5c     16: 0101 1100    \
4     4      04     24: 0000 0100    
5     109    6d     32: 0110 1101    m
6     12     0c     40: 0000 1100    
7     12     0c     48: 0000 1100    
8     24     18     56: 0001 1000    
9     65     41     64: 0100 0001    A
10    109    6d     72: 0110 1101    m
11    97     61     80: 0110 0001    a
12    122    7a     88: 0111 1010    z
13    105    69     96: 0110 1001    i
14    110    6e    104: 0110 1110    n
15    103    67    112: 0110 0111    g
16    87     57    120: 0101 0111    W
17    111    6f    128: 0110 1111    o
18    114    72    136: 0111 0010    r
19    108    6c    144: 0110 1100    l
20    100    64    152: 0110 0100    d
21    0      00    160: 0000 0000     


- 0: bitStream.Put(GSFRequestMessage == null) => False (0) => serialize deeper
- 1: bitStream.Put(MessageHeader == null) => False (0) => serialize deeper

flags = 0

- 2: compressor.Put(this, MessageHeader.flags, 4) => bs.Put(i < w) => True (1)
- 3: bs.Put(j < i) => no more bits needed => False (0)
- 4: flags => (0)
- 5: flags => (0)
- 6: flags => (0)
- 7: flags => (0)

svcClass = 18

- 8: compressor.Put(this, MessageHeader.svcClass, 4) => bs.Put(i < w) => True (1)
- 9: bs.Put(j < i) => extra 4 bits => True (1)
- 10: bs.Put(j < i) => no more bits needed => False (0)
     - bs.PutByte(0xFF & (val >> 0))
- 11: svcClass => (0)
- 12: svcClass => (0)
- 13: svcClass => (0)
- 14: svcClass => (1)
- 15: svcClass => (0)
- 16: svcClass => (0)
- 17: svcClass => (1)
- 18: svcClass => (0)

In [7]:
# IndexBit is not 0, so part of this int will go to the next byte
print_i([18])

print('\n current byte')
print_i([18 >> 3])

print('\n next byte')
print_i([(18 & (0xFF >> 5)) << 5])

bit   int    hex           bin       char
1     18     12      0: 0001 0010    

 current byte
bit   int    hex           bin       char
1     2      02      0: 0000 0010    

 next byte
bit   int    hex           bin       char
1     64     40      0: 0100 0000    @


msgType = 566

- 19: compressor.Put(this, MessageHeader.msgType, 4) => bs.Put(i < w) => True (1)
- 20: bs.Put(j < i) => extra 4 bits => True (1)
- 21: bs.Put(j < i) => extra 8 bits => True (1)
- 22: bs.Put(j < i) => no more bits needed => False (0)
- 23: msgType first it => (0)
- 24: msgType first it => (0)
- 25: msgType first it => (0)
- 26: msgType first it => (0)
- 27: msgType first it => (0)
- 28: msgType first it => (0)
- 29: msgType first it => (1)
- 30: msgType first it => (0)
- 31: msgType second it => (0)
- 32: msgType second it => (0)
- 33: msgType second it => (1)
- 34: msgType second it => (1)
- 35: msgType second it => (0)
- 36: msgType second it => (1)
- 37: msgType second it => (1)
- 38: msgType second it => (0)

In [8]:
print_i([566])
print('\n first iteration')
print_i([566 >> 8])
print('\n second iteration')
print_i([0xFF & 566])

bit   int    hex           bin       char
1     566    236      0: 1000 110110    

 first iteration
bit   int    hex           bin       char
1     2      02      0: 0000 0010    

 second iteration
bit   int    hex           bin       char
1     54     36      0: 0011 0110    6


- is_service = flags & 2 == 0
- is_response = is_service & (flags & 1) != 0
- is_request = is_service & (not is_response)
- is_notify = flags & 2 != 0
- is_discardable = flags & 16 != 0
- if IsService then write requestId integer
- if IsRequest then write logCorrelator string

requestId = 1
- 39: compressor.Put(this, MessageHeader.requestId, 4) => bs.Put(i < w) => True (1)
- 40: bs.Put(j < i) => no more bits needed => False (0)
- 41: requestId => (0)
- 42: requestId => (0)
- 43: requestId => (0)
- 44: requestId => (1)

logCorrelator = ""
- 45: compressor.Put(this, logCorrelator.UTF8.Length, 4) => bs.Put(i < w) => True (1)
- 46: bs.Put(j < i) => no more bits needed => False (0)
- 47: length => (0)
- 48: length => (0)
- 49: length => (0)
- 50: length => (0)

- 51: bitStream.Put(GSFRequest == null) => False (0) => serialize deeper

clientName = "AmazingWorld" (12)
- 52: compressor.Put(this, clientName.UTF8.Length, 4) => bs.Put(i < w) => True (1)
- 53: bs.Put(j < i) => extra 8 bits => True (1)
- 54: bs.Put(j < i) => no more bits needed => False (0)
- 55: length => 0
- 56: length => 0
- 57: length => 0
- 58: length => 0
- 59: length => 1
- 60: length => 1
- 61: length => 0
- 62: length => 0
- 63: align => 0
- 64-159: AmazingWorld

## Writing Data Length

```cs
if (length > 268435455) {
    throw new ArgumentException("length is out of range");
}
if (length > 2097151) {
    writer.Write((byte)(0x80u | (0x7Fu & (length >> 21))));
}
if (length > 16383) {
    writer.Write((byte)(0x80u | (0x7Fu & (length >> 14))));
}
if (length > 127) {
    writer.Write((byte)(0x80u | (0x7Fu & (length >> 7))));
}
writer.Write((byte)(0x7Fu & length));
```

This means that if message length is < 128 bytes, then size takes 1 byte. 128 to 16382 takes 2 bytes and so on.